#### based on quick_start.ipynb by Borhanian 

In [ ]:
from benchmarking import *

## Example of simple benchmarking

In [ ]:
# following quick_start.ipynb as a guide 
# Network initialisation

# initialisation is '<config>_<site>'
# other configurations (Tab.4): aLIGO,A+,V+,K+,Voyager-CBO,Voyager-PMO,ET 
# CE configs: <stage>-<arm length>-<optimisation> = CE1/CE2-10/20/30/40-CBO/PMO
# sites/locations (Tab.3): H,L,V,K,I,ET1,ET2,ET3,C,N,S; the latter three are Main, North, and South for CE

# choose the desired detectors
# CE-N 40km, CE-S 40km:
# network_spec = ['CE1-40-CBO_C', 'CE1-40-CBO_S']
# network_spec = ['CE2-40-CBO_C', 'CE2-40-CBO_S'] # 2nd stage of development, compare as well
# # CE-N 40km, CE-S 20km:
# network_spec = ['CE1-40-CBO_C', 'CE1-20-PMO_S']
# network_spec = ['CE2-40-CBO_C', 'CE2-20-PMO_S'] # 2nd stage
# locs = ['C', 'S']
network_spec, locs = ['aLIGO_H', 'aLIGO_L', 'aLIGO_V'], ['H', 'L', 'V']

# initialize the network with the desired detectors
net = network.Network(network_spec)

# frequency range
f = np.arange(5., 61.5, 2**-4) # i.e. looking for CBCs around 50 Hz, change for PMO

# choose the desired waveform 
wf_model_name = 'tf2' # TaylorF2, coded in gwbench, allows for symbolic differentiation
wf_other_var_dic = None # for tf2 or tf2_tidal
# other waveforms are available (e.g. tf2_tidal) but require different injection parameters

# pass the chosen waveform to the network for initialization
net.set_wf_vars(wf_model_name=wf_model_name)

# injection parameters
# for GW150914
inj_params = {
    'Mc':    30.9,
    'eta':   0.247,
    'chi1z': 0,
    'chi2z': 0,
    'DL':    475,
    'tc':    0,
    'phic':  0,
    'iota':  PI/4,
    'ra':    PI/4,
    'dec':   PI/4,
    'psi':   PI/4,
    'gmst0': 0}

# assign with respect to which parameters to take derivatives, for the FIM
deriv_symbs_string = 'Mc eta DL tc phic iota ra dec psi'

# assign which parameters to convert to cos or log versions
conv_cos = ('iota', 'dec')
conv_log = ('Mc', 'DL')

# choose whether to take Earth's rotation into account
use_rot = 0
only_net = 1

# pass all these variables to the network
net.set_net_vars(
    f=f, inj_params=inj_params,
    deriv_symbs_string=deriv_symbs_string,
    conv_cos=conv_cos, conv_log=conv_log,
    use_rot=use_rot
    )
print('Network initialised')

In [ ]:
# GW benchmarking
# symbolic derivatives (faster)
generate_symbolic_derivatives(wf_model_name, wf_other_var_dic, deriv_symbs_string, locs, use_rot)

# compute the WF polarizations and their derivatives
net.calc_wf_polarizations()
# --- numerical differentiation ---
# net.calc_wf_polarizations_derivs_num()
# --- symbolic differentiation ---
net.load_wf_polarizations_derivs_sym()
net.calc_wf_polarizations_derivs_sym()

# setup antenna patterns, location phase factors, and PSDs
net.setup_ant_pat_lpf_psds()
# results are accessed like this
# net.detectors[0].Fp # [i] for ith detector in network
# net.detectors[0].Fc # --> not frequency dependent?
# net.detectors[0].Flp
# net.detectors[0].psd # f in future calculations truncated to match psd

# compute the detector responses and their derivatives
# analogous to WF calculation
net.calc_det_responses()
# --- numerical differentiation ---
# net.calc_det_responses_derivs_num()
# --- symbolic differentiation ---
net.load_det_responses_derivs_sym()
net.calc_det_responses_derivs_sym()

# access results (either way) via
# net.detectors[0].hf
# net.detectors[0].del_hf

# calculate the network and detector SNRs
net.calc_snrs(only_net=only_net)
# print(net.snr, net.snr_sq, net.detectors[0].snr, net.detectors[0].snr_sq)

# calculate the network and detector Fisher matrices, condition numbers, ...
# ... covariance matrices, error estimates, and inversion errors
net.calc_errors(only_net=only_net) # finds FIMs, then inverts to find covariance matrix and error estimates of params
# print(net.fisher, net.cond_num, net.cov, net.errs, net.inv_err)
# print(net.detectors[0].fisher, net.detectors[0].cond_num, net.detectors[0].cov,
#       net.detectors[0].errs, net.detectors[0].inv_err)

# calculate the 90%-credible sky area (in [deg]^2)
net.calc_sky_area_90(only_net=only_net)
print('\nBenchmarking complete')

In [ ]:
# print the contents of the detector objects (inside the network)
# net.print_detectors()
# print the contents of the network objects
net.print_network()

# net.get_snrs_errs_cov_fisher_inv_err_for_key(key='network')

# # check if f truncated in any of the detectors to fit the psd
# for i in range(len(net.detectors)):
#     print(np.all(net.f == net.detectors[i].f))